In [1]:
import sys
sys.path.insert(0, '..')
from selenium import webdriver
from bs4 import BeautifulSoup
from selenium.webdriver.common.by import By
import json
import sys
import pandas as pd
import re
import time

In [2]:
from src.utils import get_full_url, AuditVideo

In [18]:
class Audit(object):
    """
    Static class that includes configuration for our Audit process
    """

    #AUDIT VARIABLES
    USER_AGE = 'young' #'young' or 'old'
    FINANCE_VIDEO_TYPE = 'traditional' #'traditional', 'blockchain', 'mixed', 'unrelated'

    WATCH_BY_RATIO = False #If True, watch videos by ratio of video duration, else watch by number of seconds
    WATCH_DURATION = 10 #Number of seconds to watch each video
    WATCH_RATIO = 0.5 #Ratio of video duration to watch

    NUM_RECOMMENDATIONS = 10 #Number of recommendations to collect, very large values may throw errors

    YOUNG_SEED_AGE_VIDEO_PATH = '../data/seed/youtube/young_videos.csv'
    OLD_SEED_AGE_VIDEO_PATH = '../data/seed/youtube/old_videos.csv'
    SEED_FINANCE_VIDEO_PATH = '../data/seed/youtube/seed_videos.csv'
    # SEED_AGE_VIDEO_PATH = '../data/raw/youtube/seed_age_videos.csv'
    # SEED_FINANCE_VIDEO_PATH = '../data/raw/youtube/seed_finance_videos.csv'

    VIDEO_ID_COLUMN = 'id'

    AUDIT_RESULTS_PATH = '../data/audit/youtube/'

In [20]:
import datetime
df = pd.read_csv(Audit.SEED_FINANCE_VIDEO_PATH, index_col=0)
# Convert duration to seconds
df['duration'] = df['duration'].apply(lambda x: datetime.datetime.strptime(x, '%H:%M:%S').time())
df['duration'] = df['duration'].apply(lambda x: x.hour * 3600 + x.minute * 60 + x.second)
df

,id,title,description,tags,raw_duration,duration,cleaned_transcript,raw_transcript,comments,comment_ids,label,link,snippet
0,gFQNPmLKj1k,How to Invest for Beginners,"Hey friends, in this video I'll give you a com...","['Ali Abdaal', 'Ali abdal', 'how to get starte...",PT29M9S,1749,"Right, so let's say\nyou want to get started w...","[{'text': ""Right, so let's say\nyou want to ge...","['Interesting ❤ Thank You ❤', 'Question about ...","['UgzTvJTisgQ45X-c3Ft4AaABAg', 'UgySEPzwDA4HNs...",Traditional,www.youtube.com/watch?v=gFQNPmLKj1k,How to Invest for Beginners. Maybe youve heard...
1,GZ42PIi9bis,I asked a personal finance expert how to invest.,Go to https://squarespace.com/mattdavella to s...,"[""Matt D'Avella"", 'matt davella', 'self-develo...",PT15M48S,948,"- Investing, when most people think of it, the...","[{'text': '- Investing, when most people think...",['Huge thanks to Ramit Sethi for joining me! S...,"['Ugx7vuOGQxrEMVYfVyN4AaABAg', 'Ugx7vuOGQxrEMV...",Traditional,www.youtube.com/watch?v=GZ42PIi9bis,I asked a personal finance expert how to inves...
2,Z3I8TPYn4Z0,Traditional and Roth IRAs | Simple Steps for a...,Individual Retirement Accounts (IRAs) can hel...,"['tdameritrade', 'TD Ameritrade', 'What is an ...",PT4M35S,275,"If you're saving for retirement, IRAs, also\nk...","[{'text': ""If you're saving for retirement, IR...",['hmmmmm do I think taxes will be HIGHER or LO...,"['UgxhIPwye-LL2ICHp594AaABAg', 'Ugw_poXl_sbpan...",Traditional,www.youtube.com/watch?v=Z3I8TPYn4Z0,Traditional and Roth IRAs | Simple Steps for a...
3,ss5yImerkhg,What is a 401(k)?,A 401(k) is an employer-sponsored retirement a...,"['401k', '401(k) contributions', '401k limit',...",PT5M23S,323,One of the most common ways people save for\nr...,[{'text': 'One of the most common ways people ...,['so if the market crashes what happens to you...,"['UgxvZb1p0tmadP_Lrn94AaABAg', 'UgxvZb1p0tmadP...",Traditional,www.youtube.com/watch?v=ss5yImerkhg,What is a 401(k)?. One of the most common ways...
4,DPnXJpDOAHI,How IRAs Work And Why They Are More Popular Th...,Choosing the right retirement account that bes...,"['CNBC', 'business', 'news', 'finance stock', ...",PT15M13S,913,are you one of the 60 million americans. acti...,[{'text': 'are you one of the 60 million ameri...,['I honestly hope they don’t pass laws that au...,"['UgyjWWVJM9zySO3L3oJ4AaABAg', 'Ugx-U2VntiMGdX...",Traditional,www.youtube.com/watch?v=DPnXJpDOAHI,How IRAs Work And Why They Are More Popular Th...
...,...,...,...,...,...,...,...,...,...,...,...,...,...
97,KgH4rDtxqHg,The Craziest Wallstreetbets Trades You Never H...,Subscribe to The Daily Upside with the link be...,NaN,PT11M56S,716,this video was made in partnership with. the ...,[{'text': 'this video was made in partnership ...,['Subscribe to The Daily Upside using the link...,"['UgwyymS09XRqCTVsajh4AaABAg', 'UgwyymS09XRqCT...",None,www.youtube.com/watch?v=KgH4rDtxqHg,The Craziest Wallstreetbets Trades You Never H...
98,p7HKvqRI_Bo,How does the stock market work? - Oliver Elfen...,"Download a free audiobook version of ""The Rich...","['TED', 'TED-Ed', 'TED Ed', 'Teded', 'Ted Educ...",PT4M30S,270,In the 1600s the Dutch East India Company \nem...,"[{'text': 'In the 1600s', 'start': 6.66, 'dura...",['Interested in pursuing your own financial go...,"['Ugy6cnjIoNBjPkCxhA94AaABAg', 'Ugy6cnjIoNBjPk...",Traditional,www.youtube.com/watch?v=p7HKvqRI_Bo,How does the stock market work? - Oliver Elfen...
99,MiybniIIvx0,Options Trading: Understanding Option Prices,📣 FREE OPTIONS TRADING MASTERCLASS | https://s...,"['stocks', 'options', 'nasdaq', 'dow', 'LNKD',...",PT7M31S,451,now that you understand why options. exist I'...,[{'text': 'now that you understand why options...,"['🚨 Hey all, thanks so much for the great feed...","['UgwGGaSpuV3IgNn9M8h4AaABAg', 'UgwGGaSpuV3IgN...",Traditional,www.youtube.com/watch?v=MiybniIIvx0,Options Trading: Understanding Option Prices. ...
100,kHybf1aC-jE,What is a Blockchain? (Animated + Examples),"In this video, we are

In [21]:
#from config import audit

def get_age_seed_videos():
    if Audit.USER_AGE == 'young':
        df = pd.read_csv(Audit.YOUNG_SEED_AGE_VIDEO_PATH, index_col=0)
    elif Audit.USER_AGE == 'old':
        df = pd.read_csv(Audit.OLD_SEED_AGE_VIDEO_PATH, index_col=0)
    else:
        raise ValueError('User age must be "young" or "old"')
    
    # Convert duration to seconds
    df['duration'] = df['duration'].apply(lambda x: datetime.datetime.strptime(x, '%H:%M:%S').time())
    df['duration'] = df['duration'].apply(lambda x: x.hour * 3600 + x.minute * 60 + x.second)
    durations = df['duration'].values
    
    #df = df[df['label'] == Audit.USER_AGE]
    video_ids = df[Audit.VIDEO_ID_COLUMN].values
    video_urls = [get_full_url(video_id) for video_id in video_ids]
    return video_urls, durations

def get_finance_seed_videos():
    df = pd.read_csv(Audit.SEED_FINANCE_VIDEO_PATH, index_col=0)
    #df = df[df['label'] == Audit.FINANCE_VIDEO_TYPE]

    # Convert duration to seconds
    df['duration'] = df['duration'].apply(lambda x: datetime.datetime.strptime(x, '%H:%M:%S').time())
    df['duration'] = df['duration'].apply(lambda x: x.hour * 3600 + x.minute * 60 + x.second)
    durations = df['duration'].values

    video_ids = df[Audit.VIDEO_ID_COLUMN].values
    video_urls = [get_full_url(video_id) for video_id in video_ids]
    return video_urls, durations

def process_durations_list(durations):
    if Audit.WATCH_BY_RATIO:
        durations = [int(duration * Audit.WATCH_RATIO) for duration in durations]
    else:
        durations = [Audit.WATCH_DURATION for duration in durations]
    return durations

get_age_seed_videos()

(['https://www.youtube.com/watch?v=W3CDMcWlPyM',
  'https://www.youtube.com/watch?v=-RQdkW-em-g',
  'https://www.youtube.com/watch?v=qN_GMtFhlo4',
  'https://www.youtube.com/watch?v=5TQhslgMt0U',
  'https://www.youtube.com/watch?v=N2EYdmYd4V8',
  'https://www.youtube.com/watch?v=wm6Ld8XcdGc',
  'https://www.youtube.com/watch?v=pWg6aUl0ldg',
  'https://www.youtube.com/watch?v=W8lJMJNOiAI',
  'https://www.youtube.com/watch?v=hSCBHRwFiqM',
  'https://www.youtube.com/watch?v=0lJYviju6RY',
  'https://www.youtube.com/watch?v=gTpogtMdtPU',
  'https://www.youtube.com/watch?v=Muwc8eyxPJs',
  'https://www.youtube.com/watch?v=nmkKo8QAeh4',
  'https://www.youtube.com/watch?v=3ErErK1wRDM',
  'https://www.youtube.com/watch?v=IPUInP53HO0',
  'https://www.youtube.com/watch?v=ViXfbDN6oTI',
  'https://www.youtube.com/watch?v=lUqIPo4DDHE',
  'https://www.youtube.com/watch?v=IAwE4o6gdts',
  'https://www.youtube.com/watch?v=sIxtOqJan2M',
  'https://www.youtube.com/watch?v=3D-vUKVQqS4'],
 array([ 902,  520,

In [5]:
from src.YTDriver import YTDriver

In [22]:
def run_audit():

    #Record start time
    start_time = time.time()
    driver = YTDriver(browser='firefox', verbose=True)

    #Watch age seed videos
    age_seed_videos, age_seed_video_durations = get_age_seed_videos()
    age_seed_video_durations = process_durations_list(age_seed_video_durations)
    driver.play_list(age_seed_videos, age_seed_video_durations, homepage_interval=0, topn=Audit.NUM_RECOMMENDATIONS)
    to_csv(driver, start_time)

    #Watch finance videos
    finance_seed_videos, finance_seed_video_durations = get_finance_seed_videos()
    finance_seed_video_durations = process_durations_list(finance_seed_video_durations)
    driver.play_list(finance_seed_videos, finance_seed_video_durations, homepage_interval=10, topn=Audit.NUM_RECOMMENDATIONS)

    driver.close() #Only closes the browser, object and results are still available

    #Save results to csv
    to_csv(driver, start_time)
run_audit()

NameError: name 'YTDriver' is not defined

In [ ]:
def to_csv():
    start_time_str = str(start_time).replace('.', '') #Remove period from start time for filenames

    video_recs_df = pd.DataFrame(driver.video_recs)
    video_recs_df['Start Time'] = start_time
    video_recs_df['Age'] = Audit.USER_AGE
    video_recs_df['Finance Video Type'] = Audit.FINANCE_VIDEO_TYPE
    video_recs_df.to_csv(Audit.AUDIT_RESULTS_PATH + f'{start_time_str}-video_recs.csv', index=False)

    homepage_recs_df = pd.DataFrame(driver.homepage_recs)
    homepage_recs_df['Start Time'] = start_time
    homepage_recs_df['Age'] = Audit.USER_AGE
    homepage_recs_df['Finance Video Type'] = Audit.FINANCE_VIDEO_TYPE
    homepage_recs_df.to_csv(Audit.AUDIT_RESULTS_PATH + f'{start_time_str}-homepage_recs.csv', index=False)

In [ ]:
def scrape_video(url):
    """

    Args:
        url (str): Must be a url for a valid YouTube video
    """
    assert('youtube.com' in url.lower())
    
    driver = webdriver.Chrome()
    driver.get(url)
    time.sleep(5)
    driver.execute_script("window.scrollTo(0, document.body.scrollHeight);")
    
    elements = driver.find_elements(By.ID, 'video-title')
    out = []
    for elem in elements[:-1]: #Last element is not actually a video
        title = elem.get_attribute('innerHTML').strip()
        link = elem.find_element(By.XPATH, "./../..").get_attribute('href') #Extracts link to video
        
        row = {'title': title, 'link': link, 'source': url}
        out.append(row)
        
    print(out)
    driver.close()
    return out

def scrape_ytdriver(filepath, recursions=5):
    driver = YTDriver(browser='firefox', verbose=True)

    videos = []
    cur_videos = driver.get_homepage()
    if not cur_videos:
        cur_videos = driver.get_homepage()
    videos += cur_videos
    
    for _ in range(recursions):
        driver.play(cur_videos[0], 10)

        cur_videos = driver.get_recommendations()
        videos += cur_videos
            
    driver.close()

    video_urls = [video.url for video in videos]
    video_urls.insert(0, 'https://www.youtube.com') #Demark start of sessions
    
    list_to_csv(video_urls, filepath)
    return video_urls